###GAN With COLORS

In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import transforms
# from model import discriminator, generator
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
import torch
import torch.nn as nn
import glob

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
"""
Hyperparameter settings
"""
# Root directory for dataset
dataroot = "data/celeba"
# Number of workers for dataloader
workers = 2
# Batch size during training
batch_size = 64
# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64
# Number of channels in the training images. For color images this is 3
nc = 3
# Size of z latent vector (i.e. size of generator input)
nz = 100
# Size of feature maps in generator
ngf = 64
# Size of feature maps in discriminator
ndf = 64
# Number of training epochs
num_epochs = 500
# Learning rate for optimizers
lr = 0.0002
# Beta1 hyperparam for Adam optimizers
beta1 = 0.5
# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

loss = nn.BCELoss()

latent_size = 100

In [ ]:
import torch
"""
Network Architectures
The following are the discriminator and generator architectures
"""
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)



In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import transforms
# from model import discriminator, generator
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
import glob
import cv2
import torchvision
avgpool = nn.AdaptiveAvgPool2d((64, 64))
from PIL import Image

In [ ]:

"""
Determine if any GPUs are available
"""
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model
# G = generator().to(device)
# D = discriminator().to(device)
#
# G_optimizer = optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
# D_optimizer = optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))

"""
Image transformation and dataloader creation
Note that we are training generation and not classification, and hence
only the train_loader is loaded
"""
# Transform
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

In [ ]:
avgpool = nn.AdaptiveAvgPool2d((64, 64))
dataset = './datasets/twimg_64/'

class afhqDataset(Dataset):
    def __init__(self, root_dir=dataset):
        self.root_dir = root_dir
        self.img_dir = root_dir + '*.png'

    def __len__(self):
        return len(glob.glob(self.img_dir))

    def __getitem__(self, idx):
        image = cv2.imread(glob.glob(self.img_dir)[idx])/255.
        image = torch.Tensor(image)
        image = image.permute(2, 0, 1)
        image = avgpool(image)
        return image

train_set = afhqDataset()
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

In [ ]:
image = cv2.imread(glob.glob(dataset + '/*')[1])/255.
image = torch.Tensor(image)
image = image.permute(2, 0, 1)
image = avgpool(image)
image = image.permute(1, 2, 0)
plt.imshow(image)

In [ ]:
# Create the generator
netG = Generator(ngpu).to(device)
# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.02.
netG.apply(weights_init)
# Print the model
#print(netG)
netD = Discriminator(ngpu).to(device)
# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netD.apply(weights_init)
# Print the model
#print(netD)

In [ ]:
# Initialize BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
for epoch in range(num_epochs):
    for idx, (imgs) in enumerate(train_loader):
        idx += 1

        # Training the discriminator
        # Real inputs are actual images of the MNIST dataset
        # Fake inputs are from the generator
        # Real inputs should be classified as 1 and fake as 0
        real_inputs = imgs.to(device)
        real_outputs = netD(real_inputs)
        real_label = torch.ones(real_inputs.shape[0], 1).to(device)

        noise = (torch.rand(real_inputs.shape[0], 100, 1, 1) - 0.5) / 0.5
        noise = noise.to(device)
#         print(noise.shape)
        fake_inputs = netG(noise)
        fake_outputs = netD(fake_inputs)
        fake_label = torch.zeros(fake_inputs.shape[0], 1).to(device)

        outputs = torch.cat((real_outputs.view(-1).unsqueeze(1), fake_outputs.view(-1).unsqueeze(1)), 0)
        targets = torch.cat((real_label, fake_label), 0)

        D_loss = loss(outputs, targets)
        optimizerD.zero_grad()
        D_loss.backward()
        optimizerD.step()

        # Training the generator
        # For generator, goal is to make the discriminator believe everything is 1
        noise = (torch.rand(real_inputs.shape[0], 100, 1, 1)-0.5)/0.5
        noise = noise.to(device)
#         print(noise.shape)
        fake_inputs = netG(noise)
        fake_outputs = netD(fake_inputs)
        fake_targets = torch.ones([fake_inputs.shape[0], 1]).to(device)
        G_loss = loss(fake_outputs.view(-1).unsqueeze(1), fake_targets)
        optimizerG.zero_grad()
        G_loss.backward()
        optimizerG.step()

        if idx % 10 == 0 or idx == len(train_loader):
            print('Epoch {} Iteration {}: discriminator_loss {:.3f} generator_loss {:.3f}'.format(epoch, idx, D_loss.item(), G_loss.item()))

    if (epoch+1) % 20 == 0:
        print("save models")
        # Save the model checkpoints
        torch.save(netG.state_dict(), f'./models/micro_64_{epoch}_G.ckpt')
        torch.save(netD.state_dict(), f'./models/micro_64_{epoch}_D.ckpt')
    #     torch.save(G, 'Generator_epoch_{}.pth'.format(epoch))
    #     print('Model saved.')

In [ ]:
for i in range(0, 5):
    torch.manual_seed(i)
    noise = (torch.rand(real_inputs.shape[0], 100, 1, 1) - 0.5) / 0.5
    noise = noise.to(device)
    output = netG(noise)
    plt.imshow(output[0].permute(1, 2, 0).detach().cpu())

In [ ]:
# import torchvision.transforms as T
# transformim = T.ToPILImage()
#
# G = torch.load('midigan_19.pth')
# G.eval()
#
# for seed in range(0,5):
#     torch.manual_seed(seed)
#     noise = (torch.rand(1, latent_size)-0.5)/0.5
#     noise = noise.to(device)
#     generated = G(noise)
#     generated = generated.cpu().detach()
#     img = generated[0].reshape(image_size, image_size)
#     img = transformim(img)
#     display(img)